In [1]:
import json
from fastapi import FastAPI

from fastapi.middleware.cors import CORSMiddleware

In [2]:
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


In [3]:
app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [4]:
user_input = 'NHS Health'

In [5]:
client = MongoClient('mongodb://localhost:27017/')
# select a database
db = client['inverted_index']
mycollection = db['inverted_index']

In [6]:
words = user_input.split()

In [7]:
words

['NHS', 'Health']

In [8]:
rel_docs = set()
for word in words:
    myquery = {"word": word}
    results  = mycollection.find(myquery)
    for result in results:
        docs = result['doc_ids']
        print(word,docs)
        rel_docs.update(docs)

NHS [11, 44, 79, 133, 192, 223, 254, 259, 276, 319, 355, 379, 383, 421, 475, 516, 526, 542, 576, 630, 689, 721, 752, 757, 774, 817, 853, 857, 876, 880, 920, 1009, 1014, 1024, 1039, 1073, 1128, 1188, 1250, 1255, 1272, 1586]
Health [5, 39, 95, 110, 114, 127, 180, 226, 254, 293, 298, 323, 398, 406, 421, 457, 465, 469, 473, 477, 490, 511, 515, 526, 536, 592, 606, 610, 623, 724, 752, 790, 796, 821, 895, 902, 920, 956, 964, 967, 971, 974, 988, 1008, 1013, 1024, 1033, 1088, 1104, 1108, 1122, 1175, 1221, 1250, 1288, 1294, 1385, 1417, 1518, 1522, 1533, 1586, 1608, 1614, 1622, 1631, 1655, 1799, 1853]


In [9]:
rel_docs = list(rel_docs)

In [10]:
len(rel_docs)

103

In [11]:
documents  = []
urls = []
for idx in rel_docs:
    query = f"SELECT * FROM sites where id = {idx};"
    cursor.execute(query)
    results = cursor.fetchall()
    for row in results:
        doc_id = row[0]
        url = row[1]
        urls.append(url)
        text = row[2]
        documents.append(text)

In [12]:
tokenized_corpus = [doc.split(" ") for doc in documents]
bm25 = BM25Okapi(tokenized_corpus)


In [13]:
tokenized_query = words

In [14]:
doc_scores = bm25.get_scores(tokenized_query)
doc_scores = doc_scores.argsort()[::-1]

In [15]:
doc_scores

array([ 84,  71,  44, 102,  35,  27,  28,  29,  30,  31,  32,  33,  34,
        36,  25,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,  26,
        24,  49,  11,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
        12,  23,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  48,
        51,  50,  89,  78,  79,  80,  81,  82,  83,  85,  86,  87,  88,
        90, 101,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,  77,
        76,  75,  74,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  72,  73,   0],
      dtype=int64)

In [16]:
rel_docs[84]

967

In [17]:
out_data = {}
for idx,score_id in enumerate(doc_scores,start=1):
    doc_id = rel_docs[score_id]
    url = urls[score_id]
    text = documents[score_id][:200]
    out_data.update({idx:{'rank':idx,'doc_id':doc_id,'url':url,'text':text }})

In [18]:
len(out_data)

103

In [19]:
json_data = json.dumps(out_data)

In [20]:
json_data

'{"1": {"rank": 1, "doc_id": 967, "url": "https://www.bbc.com/news/world-europe-jersey-64798087", "text": "Jersey building contractor Camerons Ltd goes bust A major construction contractor in Jersey has become insolvent. Camerons Ltd has ceased trading with immediate effect and is in the process of commenc"}, "2": {"rank": 2, "doc_id": 876, "url": "https://www.bbc.com/news/uk-england-beds-bucks-herts-64711076", "text": "Sepsis amputee says she has no dignity in tiny Milton Keynes home A woman whose hands and legs had to be amputated when she contracted sepsis has said living in a small home gives her \\"no dignity or i"}, "3": {"rank": 3, "doc_id": 223, "url": "https://www.bbc.com/news/education-64662613", "text": "Secondary school places: How to appeal a school admissions decision Children starting secondary school in England and Wales in September 2023 have been offered school places.  What are the options if "}, "4": {"rank": 4, "doc_id": 511, "url": "https://www.bbc.com/news/uk-eng

In [136]:
doc_id = rel_docs[84]

In [137]:
doc_id

967

In [138]:
query = f"SELECT * FROM sites where id = {doc_id};"
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

(967, 'https://www.bbc.com/news/world-europe-jersey-64798087', 'Jersey building contractor Camerons Ltd goes bust A major construction contractor in Jersey has become insolvent. Camerons Ltd has ceased trading with immediate effect and is in the process of commencing formal proceedings to wind down its business. Its board of directors said a "period of financial difficulty" had been caused by many factors, including Brexit and the Covid-19 pandemic. The company had built multiple finance buildings, schools and other properties during its 66 years of work. A statement provided by the Garenne Group, the parent company of Camerons Ltd, said rising interest rates, supply chain issues, materials and labour shortages had also caused it to cease trading. It said: "Having exhausted the options available to it to try and renegotiate the terms of a number of problematic contracts without success, and having consulted with relevant third parties and with government, the Camerons\' board feels tha

In [131]:
documents[3]

"Latvia media guide Latvia's media scene is diverse, but reflects the division of society into Latvian- and Russian-speaking audiences. Ownership of the most popular media outlets is highly concentrated. The print media have lost readers to online outlets. Television is the most popular platform, although viewing figures are declining. TV and the sector is dominated by the top commercial station TV3 and two public networks  International watchdogs rate the media environment as free, but point to difficulties in accessing trustworthy information in Russian. The invasion of Ukraine led to the blocking of dozens of Russian media outlets in Latvia.  However the presence of a large Russian-speaking community means the country remains vulnerable to Kremlin disinformation and other forms of information aggression. Some Russian outlets have moved to Latvia in search of media freedom but liberal-leaning Dozhd TV's broadcasting licence in Latvia was withdrawn. Public Latvian Radio (LR) dominates

## TESTING WORD2VWC

In [22]:
query = f"SELECT content FROM sites;"
cursor.execute(query)
results = cursor.fetchall()


In [25]:
(results)

[('American Football  Minnesota Vikings wide receiver KJ Osborn describes how he "saved a man\'s life" after pulling a driver from a burning car. Find out more about our BBC Sport App Help & FAQs',),
 ('Timeline of Nicola Payne disappearance Mother-of-one Nicola Payne was 18 when she went missing while walking to her parents\' house in Coventry in December 1991. The case has remained open ever since - here are the key moments in the long-running saga: While walking to her parents\' Henley Green house from the Winston Avenue home she shares with her boyfriend half a mile away, Nicola goes missing. Years pass without her being found, despite a police investigation and appeals by the family. Her parents, Marilyn and John Payne, make a new appeal for information, although her mother says she is unlikely to be found alive. Police say the case will remain open until they find out what happened. A series of adverts are displayed on lorries in an appeal for information. The case features on Th

In [26]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\iq\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [29]:
sw = stopwords.words('English') # this is a list of stopwords
# Function to remove stopwords per line if they are present in the line
def rem_stops_line(line, words):
    if len(line) >1:
        return [w for w in line if w not in words]
    else:
        return line

# Remove stop words for an entire text. Separate functions make it easier to parallelize if required.
def remove_stops(text, words = sw):
    return [rem_stops_line(line, words) for line in text]

In [43]:
lines = [line[0].rstrip('\n').lower() for line in results]

In [44]:
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]
lines = [word_tokenize(line) for line in lines]
filtered_lines = remove_stops(text = lines, words = sw)

In [45]:
w = w2v(
    filtered_lines,
    min_count=3,
    sg = 1,       # 1 for skip-gram, 0 for cbow
    window=7   # sliding window size
)


In [47]:
print(w.wv.most_similar('nhs'))


[('patients', 0.7442098259925842), ('hospitals', 0.736470103263855), ('dental', 0.7025613188743591), ('dentist', 0.702226459980011), ('cadwaladr', 0.6881837248802185), ('gp', 0.6866319179534912), ('lists', 0.6826653480529785), ('patient', 0.673258364200592), ('consultant', 0.6660375595092773), ('betsi', 0.6644423007965088)]
